You can download the `requirements.txt` for this course from the workspace of this lab. `File --> Open...`

# L2: Create Agents to Research and Write an Article

In this lesson, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

In [1]:
from crewai import Agent, Task, Crew
import os
from langchain_openai import ChatOpenAI

- As a LLM for your agents, you'll be using OpenAI's `gpt-3.5-turbo`.

**Optional Note:** crewAI also allow other popular models to be used as a LLM for your Agents. You can see some of the examples at the [bottom of the notebook](#1).

In [15]:
os.environ["OPENAI_API_KEY"] = "dummy_key"

llm = ChatOpenAI(
    model="ollama/llama3:latest",
    base_url="http://localhost:11434/v1"
)

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [16]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    llm=llm,
    allow_delegation=False,
	verbose=True
)

### Agent: Writer

In [17]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    llm=llm,
    allow_delegation=False,
    verbose=True
)

### Agent: Editor

In [18]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    llm=llm,
    allow_delegation=False,
    verbose=True
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [19]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [20]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

### Task: Edit

In [21]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution. 

In [22]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=True
)

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [23]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: a4f7ea75-f422-412b-8900-5b6730f77ed2                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Content Planner                                                                                         │
│                                                                                                                 │
│  Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.            │
│  2. Identify the target audience, considering their interests and pain points.                                  │
│  3. Develop a detailed content outline including an introduction, key points, and a call to action.             │
│  4. Include SEO keywords and relevant data or sources.                                                          │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Content Planner                                                                                         │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  Thought: I now can give a great answer                                                                         │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: fb93e6a9-ed1e-4ef4-91de-337af169fab2                                                                     │
│  Agent: Content Planner                                                                                         │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Content Writer                                                                                          │
│                                                                                                                 │
│  Task: 1. Use the content plan to craft a compelling blog post on Artificial Intelligence.                      │
│  2. Incorporate SEO keywords naturally.                                                                         │
│  3. Sections/Subtitles are properly named in an engaging manner.                                                │
│  4. Ensure the post is structured with an engaging introduction, insightful body, and a summarizing             │
│  conclusion.                                                                                                    │
│  5. Proofread for grammatical errors and alignment with the brand's voice.                                      │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: fbea9818-df77-4b0b-946e-0ab253d26b11                                                                     │
│  Agent: Content Writer                                                                                          │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Editor                                                                                                  │
│                                                                                                                 │
│  Task: Proofread the given blog post for grammatical errors and alignment with the brand's voice.               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Editor                                                                                                  │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  **The Rise of Artificial Intelligence: A Game-Changer or Just Another Fad?**                                   │
│  ==================================================================                                             │
│                                                                                                                 │
│  **Introduction**                                                                                               │
│  --------------                                                                                                 │
│                                                                                                                 │
│  As we navigate the complexities of the modern world, one topic that has garnered significant attention is      │
│  Artificial Intelligence (AI). AI has been touted as a revolutionary technology that has the potential to       │
│  transform industries and revolutionize the way we live. But what lies beneath this hype? Is AI just another    │
│  fad or a game-changer? In this post, we'll delve into the world of AI, exploring its benefits, challenges,     │
│  and what it means for our future.                                                                              │
│                                                                                                                 │
│  **The Benefits of Artificial Intelligence**                                                                    │
│  ------------------------------------                                                                           │
│                                                                                                                 │
│  AI has already shown tremendous promise in various fields. For instance, AI-powered chatbots have improved     │
│  customer service, while AI-driven analytics have enabled businesses to make data-driven decisions. In          │
│  healthcare, AI-assisted diagnostic tools have increased accuracy and reduced the time it takes to diagnose     │
│  diseases. Moreover, AI has also made significant contributions to finance, transportation, and education. By   │
│  leveraging AI's capabilities, we can streamline processes, enhance decision-making, and drive innovation.      │
│                                                                                                                 │
│  **The Challenges of Artificial Intelligence**                                                                  │
│  --------------------------------------------                                                                   │
│                                                                                                                 │
│  While AI has many benefits, there are also concerns surrounding its adoption. One of the primary challenges    │
│  is job displacement. As AI takes over repetitive and mundane tasks, there's a risk that human workers may      │
│  lose their jobs. Additionally, there are ethical considerations to be taken into account. For instance,        │
│  AI-powered facial recognition systems have raised privacy concerns and questions about bias.                   │
│                                                                                                                 │
│  **The Future of Artificial Intelligence**             

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 5e407281-fb6e-4212-9088-401ca048e0a5                                                                     │
│  Agent: Editor                                                                                                  │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: a4f7ea75-f422-412b-8900-5b6730f77ed2                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: **The Rise of Artificial Intelligence: A Game-Changer or Just Another Fad?**                     │
│  ==================================================================                                             │
│                                                                                                                 │
│  **Introduction**                                                                                               │
│  --------------                                                                                                 │
│                                                                                                                 │
│  As we navigate the complexities of the modern world, one topic that has garnered significant attention is      │
│  Artificial Intelligence (AI). AI has been touted as a revolutionary technology that has the potential to       │
│  transform industries and revolutionize the way we live. But what lies beneath this hype? Is AI just another    │
│  fad or a game-changer? In this post, we'll delve into the world of AI, exploring its benefits, challenges,     │
│  and what it means for our future.                                                                              │
│                                                                                                                 │
│  **The Benefits of Artificial Intelligence**                                                                    │
│  ------------------------------------                                                                           │
│                                                                                                                 │
│  AI has already shown tremendous promise in various fields. For instance, AI-powered chatbots have improved     │
│  customer service, while AI-driven analytics have enabled businesses to make data-driven decisions. In          │
│  healthcare, AI-assisted diagnostic tools have increased accuracy and reduced the time it takes to diagnose     │
│  diseases. Moreover, AI has also made significant contributions to finance, transportation, and education. By   │
│  leveraging AI's capabilities, we can streamline processes, enhance decision-making, and drive innovation.      │
│                                                                                                                 │
│  **The Challenges of Artificial Intelligence**                                                                  │
│  --------------------------------------------                                                                   │
│                                                                                                                 │
│  While AI has many benefits, there are also concerns surrounding its adoption. One of the primary challenges    │
│  is job displacement. As AI takes over repetitive and mundane tasks, there's a risk that human workers may      │
│  lose their jobs. Additionally, there are ethical considerations to be taken into account. For instance,        │
│  AI-powered facial recognition systems have raised privacy concerns and questions about bias.                   │
│                                                       

- Display the results of your execution as markdown in the notebook.

In [29]:
from IPython.display import Markdown
Markdown(result.raw)

**The Rise of Artificial Intelligence: A Game-Changer or Just Another Fad?**
==================================================================

**Introduction**
--------------

As we navigate the complexities of the modern world, one topic that has garnered significant attention is Artificial Intelligence (AI). AI has been touted as a revolutionary technology that has the potential to transform industries and revolutionize the way we live. But what lies beneath this hype? Is AI just another fad or a game-changer? In this post, we'll delve into the world of AI, exploring its benefits, challenges, and what it means for our future.

**The Benefits of Artificial Intelligence**
------------------------------------

AI has already shown tremendous promise in various fields. For instance, AI-powered chatbots have improved customer service, while AI-driven analytics have enabled businesses to make data-driven decisions. In healthcare, AI-assisted diagnostic tools have increased accuracy and reduced the time it takes to diagnose diseases. Moreover, AI has also made significant contributions to finance, transportation, and education. By leveraging AI's capabilities, we can streamline processes, enhance decision-making, and drive innovation.

**The Challenges of Artificial Intelligence**
--------------------------------------------

While AI has many benefits, there are also concerns surrounding its adoption. One of the primary challenges is job displacement. As AI takes over repetitive and mundane tasks, there's a risk that human workers may lose their jobs. Additionally, there are ethical considerations to be taken into account. For instance, AI-powered facial recognition systems have raised privacy concerns and questions about bias.

**The Future of Artificial Intelligence**
-----------------------------------------

As AI continues to evolve, it's essential to address these challenges head-on. Governments and corporations must work together to develop policies that mitigate the negative impacts of AI adoption. Furthermore, there needs to be a focus on developing AI systems that are transparent, accountable, and fair. By doing so, we can harness the power of AI to create a brighter future for all.

**Conclusion**
----------

In conclusion, Artificial Intelligence is not just another fad; it's a game-changer. While there are challenges to be addressed, the benefits of AI far outweigh the drawbacks. As we move forward, it's crucial that we work together to develop responsible and ethical AI systems that benefit humanity as a whole.

**SEO Keywords:** Artificial Intelligence, AI, Machine Learning, Robotics, Automation, Technology

## Try it Yourself

- Pass in a topic of your choice and see what the agents come up with!

In [ ]:
topic = "YOUR TOPIC HERE"
result = crew.kickoff(inputs={"topic": topic})

In [ ]:
Markdown(result)

<a name='1'></a>
 ## Other Popular Models as LLM for your Agents

#### Hugging Face (HuggingFaceHub endpoint)

```Python
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    huggingfacehub_api_token="<HF_TOKEN_HERE>",
    task="text-generation",
)

### you will pass "llm" to your agent function
```

#### Mistral API

```Python
OPENAI_API_KEY=your-mistral-api-key
OPENAI_API_BASE=https://api.mistral.ai/v1
OPENAI_MODEL_NAME="mistral-small"
```

#### Cohere

```Python
from langchain_community.chat_models import ChatCohere
# Initialize language model
os.environ["COHERE_API_KEY"] = "your-cohere-api-key"
llm = ChatCohere()

### you will pass "llm" to your agent function
```

### For using Llama locally with Ollama and more, checkout the crewAI documentation on [Connecting to any LLM](https://docs.crewai.com/how-to/LLM-Connections/).